
## Temporização do sensor e do atuador

Na lição 3, você construiu uma luz noturna - um LED que acende assim que um nível baixo de luz é detectado por um sensor de luz. O sensor de luz detectou uma mudança nos níveis de luz instantaneamente, e o dispositivo foi capaz de responder rapidamente, limitado apenas pela duração do atraso na função `loop` ou loop `while True:`. Como um desenvolvedor de IoT, você nem sempre pode confiar em um loop de feedback tão rápido.

### Tempo para umidade do solo

Se você fez a última lição sobre umidade do solo usando um sensor físico, você deve ter notado que levou alguns segundos para a leitura de umidade do solo cair depois que você regou sua planta. Isso não é porque o sensor é lento, mas porque leva tempo para a água penetrar no solo.

> 💁 Se você regou muito perto do sensor, pode ter visto a leitura cair rapidamente e depois subir novamente - isso é causado pela água perto do sensor se espalhando pelo resto do solo, reduzindo a umidade do solo pelo sensor.

![Uma medição de umidade do solo de 658 não muda durante a rega, ela só cai para 320 após a rega, quando a água penetra no solo](https://github.com/microsoft/IoT-For-Beginners/raw/main/images/soil-moisture-travel.png)

No diagrama acima, uma leitura de umidade do solo mostra 658. A planta é regada, mas essa leitura não muda imediatamente, pois a água ainda não chegou ao sensor. A regação pode até terminar antes que a água chegue ao sensor e o valor caia para refletir o novo nível de umidade.

Se você estivesse escrevendo um código para controlar um sistema de irrigação por meio de um relé com base nos níveis de umidade do solo, precisaria levar esse atraso em consideração e criar um tempo mais inteligente no seu dispositivo de IoT.

✅ Reserve um momento para pensar em como você pode fazer isso.



### Sensor de controle e tempo do atuador

Imagine que você foi encarregado de construir um sistema de irrigação para uma fazenda. Com base no tipo de solo, o nível ideal de umidade do solo para as plantas cultivadas foi encontrado para corresponder a uma leitura de voltagem analógica de 400-450.

Você pode programar o dispositivo da mesma forma que a luz noturna - toda vez que o sensor lê acima de 450, ligue um relé para ligar uma bomba. O problema é que a água demora um pouco para sair da bomba, passar pelo solo até o sensor. O sensor interromperá a água quando detectar um nível de 450, mas o nível da água continuará caindo à medida que a água bombeada continua encharcando o solo. O resultado final é desperdício de água e o risco de danos às raízes.

✅ Lembre-se: muita água pode ser tão ruim para as plantas quanto pouca, além de desperdiçar um recurso precioso.

A melhor solução é entender que há um atraso entre o acionamento do atuador e a mudança da propriedade que o sensor lê. Isso significa que não apenas o sensor deve esperar um pouco antes de medir o valor novamente, mas o atuador precisa desligar por um tempo antes que a próxima medição do sensor seja feita.

**Por quanto tempo o relé deve ficar ligado a cada vez?**
    * É melhor pecar pelo excesso de cautela e ligar o relé apenas por um curto período
    * Depois esperar a água penetrar e verificar novamente os níveis de umidade. 
    * Afinal, você sempre pode ligá-lo novamente para adicionar mais água, você não pode remover água do solo.

> 💁 Esse tipo de controle de tempo é muito específico para o dispositivo IoT que você está construindo, a propriedade que você está medindo e os sensores e atuadores usados.

![Uma planta de morango conectada à água por meio de uma bomba, com a bomba conectada a um relé. O relé e um sensor de umidade do solo na planta estão ambos conectados a um Raspberry Pi](https://github.com/microsoft/IoT-For-Beginners/raw/main/images/strawberry-with-pump.png)

Por exemplo, eu tenho uma planta de morango com um sensor de umidade do solo e uma bomba controlada por um relé. Observei que quando adiciono água, leva cerca de 20 segundos para a leitura de umidade do solo se estabilizar. Isso significa que preciso desligar o relé e esperar 20 segundos antes de verificar os níveis de umidade. Prefiro ter pouca água do que muita - sempre posso ligar a bomba novamente, mas não posso tirar água da planta.

![Etapa 1, faça a medição. Etapa 2, adicione água. Etapa 3, espere a água penetrar no solo. Etapa 4, faça a medição novamente](https://github.com/microsoft/IoT-For-Beginners/raw/main/images/soil-moisture-delay.png)

Isso significa que o melhor processo seria um ciclo de regação mais ou menos assim:

* Ligue a bomba por 5 segundos
* Aguarde 20 segundos
* Verifique a umidade do solo
* Se o nível ainda estiver acima do que preciso, repita os passos acima

5 segundos podem ser muito tempo para a bomba, especialmente se os níveis de umidade estiverem apenas um pouco acima do nível necessário. A melhor maneira de saber qual tempo usar é experimentá-lo e, em seguida, ajustar quando tiver dados do sensor, com um loop de feedback constante. Isso pode até levar a um tempo mais granular, como ligar a bomba por 1s para cada 100 acima da umidade do solo necessária, em vez de 5 segundos fixos.

✅ Faça alguma pesquisa: Há outras considerações de tempo? A planta pode ser regada a qualquer momento em que a umidade do solo estiver muito baixa, ou há horários específicos do dia que são bons e ruins para regar as plantas?

> 💁 As previsões meteorológicas também podem ser levadas em consideração ao controlar sistemas de irrigação automatizados para cultivo ao ar livre. Se houver previsão de chuva, a irrigação pode ser suspensa até que a chuva acabe. Nesse ponto, o solo pode estar úmido o suficiente para não precisar de irrigação, muito mais eficiente do que desperdiçar água regando logo antes da chuva.



## Adicione tempo ao seu servidor de controle de planta

O código do servidor pode ser modificado para adicionar controle em torno do tempo do ciclo de irrigação e esperar que os níveis de umidade do solo mudem. A lógica do servidor para controlar o tempo do relé é:

1. Mensagem de telemetria recebida
1. Verifique o nível de umidade do solo
1. Se estiver ok, não faça nada. 
1. Se a leitura estiver muito alta (significando que a umidade do solo está muito baixa), então:
    1. Envie um comando para ligar o relé
    1. Aguarde 5 segundos
    1. Envie um comando para desligar o relé
    1. Aguarde 20 segundos para que os níveis de umidade do solo se estabilizem

O ciclo de irrigação, o processo desde o recebimento da mensagem de telemetria até estar pronto para processar os níveis de umidade do solo novamente, leva cerca de 25 segundos. Estamos enviando os níveis de umidade do solo a cada 10 segundos, então há uma sobreposição onde uma mensagem é recebida enquanto o servidor está esperando os níveis de umidade do solo se estabilizarem, o que poderia iniciar outro ciclo de irrigação.

Há duas opções para contornar isso:

* Altere o código do dispositivo IoT para enviar telemetria apenas a cada minuto, dessa forma o ciclo de irrigação será concluído antes que a próxima mensagem seja enviada
* Cancelar a assinatura da telemetria durante o ciclo de rega

A primeira opção nem sempre é uma boa solução para grandes fazendas. O fazendeiro pode querer capturar os níveis de umidade do solo enquanto o solo está sendo regado para análise posterior, por exemplo, para estar ciente do fluxo de água em diferentes áreas da fazenda para orientar uma irrigação mais direcionada. A segunda opção é melhor - o código está apenas ignorando a telemetria quando não pode usá-la, mas a telemetria ainda está lá para outros serviços que podem assiná-la.

> 💁 Os dados de IoT não são enviados de apenas um dispositivo para apenas um serviço, em vez disso, muitos dispositivos podem enviar dados para um corretor, e muitos serviços podem ouvir os dados do corretor. Por exemplo, um serviço pode ouvir os dados de umidade do solo e armazená-los em um banco de dados para análise posterior. Outro serviço também pode ouvir a mesma telemetria para controlar um sistema de irrigação.



### Tarefa - adicionar tempo ao seu servidor de controle de planta

Atualize o código do seu servidor para executar o relé por 5 segundos e depois aguarde 20 segundos.

1. Abra a pasta `soil-moisture-sensor-server` no VS Code se ela ainda não estiver aberta. Certifique-se de que o ambiente virtual esteja ativado.

1. Abra o arquivo `app.py`

1. Adicione o seguinte código ao arquivo `app.py` abaixo das importações existentes:

    ```python
    import threading
    ```

    Esta instrução importa `threading` de bibliotecas Python, o threading permite que o Python execute outro código enquanto espera.

1. Adicione o seguinte código antes da função `handle_telemetry` que manipula mensagens de telemetria recebidas pelo código do servidor:

    ```python
    water_time = 5
    wait_time = 20
    ```

    Isso define por quanto tempo o relé deve ser executado (`water_time`) e quanto tempo esperar depois para verificar a umidade do solo (`wait_time`).

1. Abaixo deste código, adicione o seguinte:

    ```python
    def send_relay_command(client, state):
        command = { 'relay_on' : state }
        print("Sending message:", command)
        client.publish(server_command_topic, json.dumps(command))
    ```

    Este código define uma função chamada `send_relay_command` que envia um comando por MQTT para controlar o relé. A telemetria é criada como um dicionário e, em seguida, convertida em uma string JSON. O valor passado para `state` determina se o relé deve estar ligado ou desligado.

1. Após a função `send_relay_code`, adicione o seguinte código:

    ```python
    def control_relay(client):
        print("Unsubscribing from telemetry")
        mqtt_client.unsubscribe(client_telemetry_topic)

        send_relay_command(client, True)
        time.sleep(water_time)
        send_relay_command(client, False)

        time.sleep(wait_time)

        print("Subscribing to telemetry")
        mqtt_client.subscribe(client_telemetry_topic)
    ```

    Isso define uma função para controlar o relé com base no tempo necessário. Ele começa cancelando a assinatura da telemetria para que as mensagens de umidade do solo não sejam processadas enquanto a irrigação estiver acontecendo. Em seguida, ele envia um comando para ligar o relé. Ele então espera pelo `water_time` antes de enviar um comando para desligar o relé. Finalmente, ele espera que os níveis de umidade do solo se estabilizem por `wait_time` segundos. Ele então se inscreve novamente na telemetria.

1. Altere a função `handle_telemetry` para o seguinte:

    ```python
    def handle_telemetry(client, userdata, message):
        payload = json.loads(message.payload.decode())
        print("Message received:", payload)

        if payload['soil_moisture'] > 450:
            threading.Thread(target=control_relay, args=(client,)).start()
    ```

    Este código verifica o nível de umidade do solo. Se for maior que 450, o solo precisa ser regado, então ele chama a função `control_relay`. Esta função é executada em um thread separado, em execução em segundo plano.

In [1]:
import time
import random
import json
import paho.mqtt.client as mqtt

import threading

water_time = 5
wait_time = 20

# Configuração do broker MQTT
broker = 'mqtt.eclipseprojects.io'  # Endereço do broker MQTT
port = 1883  # Porta padrão do MQTT
telemetry_topic = "capacitacao-iot/telemetria"  # Tópico para envio e recebimento de telemetria
commands_topic = "capacitacao-iot/commandos"  # Tópico para envio de comandos

# Gera um identificador único para o cliente MQTT
client_id = f'capacitacao-iot-mqtt-nightlight_client-{random.randint(0, 100000)}'
print("Client ID: " + client_id)

def send_relay_command(client, state):
    command = { 'relay_on' : state }
    print("Enviando comando:", command)
    client.publish(commands_topic, json.dumps(command))

def control_relay(client):
    print("Unsubscribing from telemetry")
    mqttc.unsubscribe(telemetry_topic)

    send_relay_command(client, True)
    time.sleep(water_time)
    send_relay_command(client, False)

    time.sleep(wait_time)

    print("Subscribing to telemetry")
    mqttc.subscribe(telemetry_topic)


# Função de callback chamada quando o cliente se conecta ao broker MQTT
def on_connect(client, userdata, flags, reason_code, properties):
    # Verifica se a conexão foi bem-sucedida ou houve falha
    if reason_code != 0:
        print(f"Falha ao conectar: {reason_code}. 'loop_forever()' tentará reconectar.")
    else:
        print("Conectado ao broker MQTT!")
        # Inscreve o cliente no tópico de telemetria
        client.subscribe(telemetry_topic)

# Função de callback chamada ao receber uma mensagem no tópico de telemetria
def handle_telemetry(client, userdata, message):
    payload = json.loads(message.payload.decode())
    print("Message received:", payload)

    if payload['soil_moisture'] > 450:
        threading.Thread(target=control_relay, args=(client,)).start()
    '''
    # Decodifica o payload da mensagem recebida
    payload = json.loads(message.payload.decode())
    print("Mensagem recebida:", payload)

    # Verifica o valor de luminosidade e prepara um comando para o LED
    command = {'relay_on': payload['soil_moisture'] < 450}
    print("Enviando comando:", command)

    # Publica o comando no tópico de comandos
    client.publish(commands_topic, json.dumps(command))'''


# Cria o cliente MQTT e especifica a versão da API de callback
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, client_id=client_id)

# Associa as funções de callback aos respectivos eventos
mqttc.on_connect = on_connect  # Callback para o evento de conexão
mqttc.on_message = handle_telemetry  # Callback para o evento de recebimento de mensagens

# Configura dados adicionais de usuário, se necessário
mqttc.user_data_set([])

# Conecta ao broker MQTT
mqttc.connect(broker, port)

# Inicia o loop do cliente MQTT (bloqueante), que gerencia eventos de conexão e mensagens
mqttc.loop_forever()

Client ID: capacitacao-iot-mqtt-nightlight_client-86708
Conectado ao broker MQTT!
Message received: {'soil_moisture': 389}
Message received: {'soil_moisture': 283}
Message received: {'soil_moisture': 416}
Message received: {'soil_moisture': 850}
Unsubscribing from telemetry
Enviando comando: {'relay_on': True}
Enviando comando: {'relay_on': False}
Subscribing to telemetry
Message received: {'soil_moisture': 786}
Unsubscribing from telemetry
Enviando comando: {'relay_on': True}
Enviando comando: {'relay_on': False}


KeyboardInterrupt: 


1. Certifique-se de que seu dispositivo IoT esteja funcionando, então execute este código. Altere os níveis de umidade do solo e observe o que acontece com o relé - ele deve ligar por 5 segundos e então permanecer desligado por pelo menos 20 segundos, ligando somente se os níveis de umidade do solo não forem suficientes.

    ```saída
    (.venv) ➜ servidor-sensor-de-umidade-do-solo ✗ aplicativo python.py
    Mensagem recebida: {'soil_moisture': 457}
    Cancelar inscrição da telemetria
    Enviando mensagem: {'relay_on': True}
    Enviando mensagem: {'relay_on': False}
    Assinando a telemetria
    Mensagem recebida: {'soil_moisture': 302}
    ```

    Uma boa maneira de testar isso em um sistema de irrigação simulado é usar solo seco e, em seguida, despejar água manualmente enquanto o relé estiver ligado, parando de despejar quando o relé desligar.

> 💁 Você pode encontrar esse código na pasta [code-timing](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/3-automated-plant-watering/code-timing).

> 💁 Se você quiser usar uma bomba para construir um sistema de irrigação real, então você pode usar uma [bomba de água de 6 V](https://www.seeedstudio.com/6V-Mini-Water-Pump-p-1945.html) com uma [fonte de alimentação de terminal USB](https://www.adafruit.com/product/3628). Certifique-se de que a energia de ou para a bomba esteja conectada através do relé.

---

## 🚀 Desafio

Você consegue pensar em algum outro IoT ou outro dispositivo elétrico que tenha um problema similar em que demora um pouco para os resultados do atuador chegarem ao sensor? Você provavelmente tem alguns em sua casa ou escola.

* Quais propriedades eles medem?
* Quanto tempo leva para a propriedade mudar depois que um atuador é usado?
* É aceitável que o imóvel sofra alterações além do valor exigido?
* Como ele pode ser retornado ao valor requerido, se necessário?

## Revisão e autoestudo

* Leia mais sobre relés, incluindo seu uso histórico em centrais telefônicas na [página da Wikipedia sobre relés](https://wikipedia.org/wiki/Relay).